# Online Customer Support with Handoffs

## Overview

The **handoffs pattern** is a multi-agent architecture where agents pass control to each other through state transitions. This tutorial demonstrates how to build an online customer support system where different "agents" (really, different configurations of a single agent) handle specific stages of the support workflow.

In this tutorial, you'll build a support bot that:
- Collects warranty information before proceeding
- Classifies issues as hardware or software
- Provides solutions or escalates to human support
- Maintains conversation state across multiple turns

Unlike the supervisor pattern where subagents are called as tools, handoffs create a **state machine** where the active agent changes based on workflow progress. The key insight is that each "agent" is just a different configuration (system prompt + tools) of the same underlying agent, selected dynamically based on state.

## Setup

### Installation

First, install the required package:

In [ ]:
!pip install langchain

### LangSmith Setup

Set up LangSmith to inspect what is happening inside your agent:

In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

### Initialize Chat Model

Select a chat model to use:

In [1]:
from langchain.chat_models import init_chat_model

model = init_chat_model("anthropic:claude-3-5-sonnet-latest")

ModuleNotFoundError: No module named 'langchain.chat_models'

## Step 1: Define Custom State

First, define a custom state schema that tracks which agent is currently active:

In [ ]:
from langchain.agents import AgentState
from typing_extensions import NotRequired
from typing import Literal

# Define the possible agent types
AgentType = Literal["warranty_collector", "issue_classifier", "resolution_specialist"]

class SupportState(AgentState):
    """State for customer support workflow with handoffs."""
    active_agent: NotRequired[AgentType]
    warranty_status: NotRequired[Literal["in_warranty", "out_of_warranty", "unknown"]]
    issue_type: NotRequired[Literal["hardware", "software", "unknown"]]

The `active_agent` field is the core of the handoffs pattern - it determines which agent configuration is loaded on each turn.

## Step 2: Define Agent Configurations

Create a dataclass to hold agent configurations. Each configuration specifies the system prompt and available tools for that agent:

In [ ]:
from dataclasses import dataclass
from typing import List
from langchain_core.tools import BaseTool

@dataclass
class AgentConfig:
    """Configuration for a specific agent in the handoff workflow."""
    name: AgentType
    system_prompt: str
    tools: List[BaseTool]

## Step 3: Create Handoff and Workflow Tools

Create tools that update the workflow state. These tools allow agents to record information and transition to the next agent:

In [ ]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command

@tool
def record_warranty_status(
    status: Literal["in_warranty", "out_of_warranty"],
    runtime: ToolRuntime[None, SupportState]
) -> Command:
    """Record the customer's warranty status and transition to issue classification."""
    return Command(update={
        "warranty_status": status,
        "active_agent": "issue_classifier"
    })

@tool
def record_issue_type(
    issue_type: Literal["hardware", "software"],
    runtime: ToolRuntime[None, SupportState]
) -> Command:
    """Record the type of issue and transition to resolution specialist."""
    return Command(update={
        "issue_type": issue_type,
        "active_agent": "resolution_specialist"
    })

@tool
def escalate_to_human(
    reason: str,
    runtime: ToolRuntime[None, SupportState]
) -> str:
    """Escalate the case to a human support specialist."""
    # In a real system, this would create a ticket, notify staff, etc.
    return f"Escalating to human support. Reason: {reason}"

@tool
def provide_solution(
    solution: str,
    runtime: ToolRuntime[None, SupportState]
) -> str:
    """Provide a solution to the customer's issue."""
    return f"Solution provided: {solution}"

## Step 4: Create Agent Configurations

Now define the specific configurations for each agent:

In [ ]:
AGENT_CONFIGS = {
    "warranty_collector": AgentConfig(
        name="warranty_collector",
        system_prompt="""You are a warranty verification specialist.

Your task:
1. Greet the customer warmly
2. Ask if their device is under warranty
3. Use record_warranty_status to record their response and move to the next step

Be conversational and friendly. Don't ask multiple questions at once.""",
        tools=[record_warranty_status]
    ),

    "issue_classifier": AgentConfig(
        name="issue_classifier",
        system_prompt="""You are an issue classification specialist.

Your task:
1. Ask the customer to describe their issue
2. Determine if it's a hardware issue (physical damage, broken parts) or software issue (app crashes, performance)
3. Use record_issue_type to record the classification and move to the next step

If unclear, ask clarifying questions before classifying.""",
        tools=[record_issue_type]
    ),

    "resolution_specialist": AgentConfig(
        name="resolution_specialist",
        system_prompt="""You are a technical resolution specialist.

Context available to you:
- Warranty status: {warranty_status}
- Issue type: {issue_type}

Your task:
1. For SOFTWARE issues: provide troubleshooting steps using provide_solution
2. For HARDWARE issues:
   - If IN WARRANTY: explain warranty repair process using provide_solution
   - If OUT OF WARRANTY: escalate_to_human for paid repair options

Be specific and helpful in your solutions.""",
        tools=[provide_solution, escalate_to_human]
    )
}

Note how the resolution specialist's prompt references state variables - we'll inject these dynamically.

## Step 5: Create Dynamic Configuration Middleware

Create middleware that reads the `active_agent` state and configures the agent accordingly:

In [ ]:
from langchain.agents.middleware import AgentMiddleware, ModelRequest, ModelResponse
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from typing import Callable

class HandoffMiddleware(AgentMiddleware[SupportState]):
    """Middleware that dynamically configures the agent based on active_agent state.
    
    This middleware manages agent-specific message histories to ensure each agent
    only sees relevant context, preventing unbounded history growth and confusion.
    """

    state_schema = SupportState

    def __init__(self, agent_configs: dict[AgentType, AgentConfig]):
        super().__init__()
        self.agent_configs = agent_configs

    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        """Configure system prompt, tools, and message history based on active_agent."""

        # Get the current active agent from state (default to warranty_collector)
        active_agent = request.state.get("active_agent", "warranty_collector")
        config = self.agent_configs[active_agent]

        # Build context summary from state (what previous agents learned)
        context = self._build_context_summary(request.state)
        
        # Format the system prompt with current state values
        system_prompt = config.system_prompt.format(
            warranty_status=request.state.get("warranty_status", "unknown"),
            issue_type=request.state.get("issue_type", "unknown")
        )
        
        # Inject context summary into system prompt
        if context:
            system_prompt += f"\n\n{context}"

        # Get only recent, relevant messages for this agent
        relevant_messages = self._get_recent_messages(request.messages, max_turns=2)
        
        # Construct valid message sequence: System message, then conversation
        messages = [SystemMessage(content=system_prompt)] + relevant_messages

        # Update request with new configuration
        request.messages = messages
        request.tools = config.tools

        return handler(request)
    
    def _build_context_summary(self, state: dict) -> str:
        """Build a summary of what previous agents learned from state.
        
        This allows each agent to have a clean message history while still
        having access to structured information gathered by previous agents.
        """
        parts = []
        
        if state.get("warranty_status") and state["warranty_status"] != "unknown":
            parts.append(f"The customer's warranty status has been confirmed as: {state['warranty_status']}.")
        
        if state.get("issue_type") and state["issue_type"] != "unknown":
            parts.append(f"The issue has been classified as: {state['issue_type']}.")
        
        return " ".join(parts) if parts else ""
    
    def _get_recent_messages(self, messages: list, max_turns: int = 2) -> list:
        """Get last N conversation turns, ensuring valid message sequence.
        
        A turn consists of: user message + AI response (+ optional tool messages).
        This prevents message history from growing unbounded and ensures each
        agent only sees its relevant context.
        
        Args:
            messages: Full message history
            max_turns: Maximum number of conversation turns to retain
            
        Returns:
            List of recent messages forming complete, valid turns
        """
        if not messages:
            return []
        
        # Work backwards to collect complete turns
        turns = []
        current_turn = []
        
        for msg in reversed(messages):
            if isinstance(msg, SystemMessage):
                continue  # Skip old system messages (we inject our own)
            
            current_turn.insert(0, msg)
            
            # A turn starts with a user message
            if isinstance(msg, HumanMessage):
                turns.insert(0, current_turn)
                current_turn = []
                
                if len(turns) >= max_turns:
                    break
        
        # Flatten turns into message list
        result = []
        for turn in turns:
            result.extend(turn)
        
        # Ensure we start with a user message for valid history
        # (Most LLM providers require this pattern)
        while result and not isinstance(result[0], HumanMessage):
            result.pop(0)
        
        return result

This middleware:
1. Reads `active_agent` from state to determine which agent configuration to use
2. Looks up the corresponding configuration (system prompt + tools)
3. **Builds a context summary** from state that captures what previous agents learned
4. Injects the system prompt with both state values AND the context summary
5. **Filters message history** to only include recent turns (prevents unbounded growth)
6. Ensures valid message sequences (system → user → AI → tool alternation)

### Key Innovation: Separation of State and Messages

The middleware solves a critical problem with multi-agent handoffs:

**Problem**: If all agents see the full message history, it grows unbounded and agents get confused by other agents' conversations.

**Solution**: 
- **State dict** holds structured data (warranty status, issue type) - this is the cross-agent memory
- **Message history** is agent-scoped and bounded - each agent only sees its recent conversation
- **System message** injects context summaries from state - agents know what happened without seeing full history

This means:
- The warranty_collector doesn't see issue_classifier's messages
- The resolution_specialist doesn't see warranty_collector's messages
- BUT each agent knows what was learned via state summaries in their system prompt

## Step 6: Create the Agent

Now create the agent with the handoff middleware and a checkpointer for state persistence:

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

# Collect all tools from all configurations
all_tools = []
for config in AGENT_CONFIGS.values():
    all_tools.extend(config.tools)

# Create the agent with middleware
agent = create_agent(
    model,
    tools=all_tools,
    state_schema=SupportState,
    middleware=[HandoffMiddleware(AGENT_CONFIGS)],
    checkpointer=InMemorySaver()  # Required for state persistence across turns
)

**Why a checkpointer?** The checkpointer maintains state across conversation turns. Without it, the `active_agent` state would be lost between user messages, breaking the handoff flow.

## Step 7: Test the Workflow

Test the complete handoff workflow:

In [ ]:
from langchain_core.messages import HumanMessage

# Configuration for this conversation thread
config = {"configurable": {"thread_id": "support-001"}}

# Initial message - starts with warranty_collector
print("=== Turn 1: Warranty Collection ===")
result = agent.invoke(
    {
        "messages": [HumanMessage("Hi, my phone screen is cracked")]
    },
    config
)
print(result["messages"][-1].content)

In [ ]:
# User responds about warranty
print("\n=== Turn 2: User responds about warranty ===")
result = agent.invoke(
    {
        "messages": [HumanMessage("Yes, it's still under warranty")]
    },
    config
)
print(result["messages"][-1].content)
print(f"Active agent: {result.get('active_agent')}")

In [ ]:
# User describes the issue
print("\n=== Turn 3: Issue classification ===")
result = agent.invoke(
    {
        "messages": [HumanMessage("The screen is physically cracked, it happened when I dropped it")]
    },
    config
)
print(result["messages"][-1].content)
print(f"Active agent: {result.get('active_agent')}")

In [ ]:
# Resolution
print("\n=== Turn 4: Resolution ===")
result = agent.invoke(
    {
        "messages": [HumanMessage("What should I do?")]
    },
    config
)
print(result["messages"][-1].content)

Expected flow:
1. **Warranty Collector**: Asks about warranty status
2. **Issue Classifier**: Asks about the problem, determines it's hardware
3. **Resolution Specialist**: Provides warranty repair instructions

### Turn 3: After issue classified
Tool call: `record_issue_type("hardware")` returns:
```python
Command(update={
    "issue_type": "hardware",
    "active_agent": "resolution_specialist"  # State transition!
})
```

Next turn, middleware configures:
- System prompt: Resolution specialist instructions (with state context)
- Tools: `[provide_solution, escalate_to_human]`

## Message History Management

A critical aspect of the handoffs pattern is managing what each agent sees. The middleware implements **message history scoping** to solve several problems:

### The Problem

Without message history management:
```python
# Full conversation after 3 agent transitions:
[
    HumanMessage("Hi, my phone screen is cracked"),
    AIMessage("Is your device under warranty?"),  # warranty_collector
    HumanMessage("Yes, it's still under warranty"),
    AIMessage("Tool: record_warranty_status"),     # warranty_collector
    ToolMessage("Recorded warranty status"),
    AIMessage("Can you describe the issue?"),      # issue_classifier
    HumanMessage("The screen is physically cracked"),
    AIMessage("Tool: record_issue_type"),          # issue_classifier
    ToolMessage("Recorded issue type"),
    AIMessage("I can help with warranty repair"),  # resolution_specialist
]
```

**Problems**:
- ❌ Message history grows unbounded (9+ messages after just 3 turns)
- ❌ Resolution specialist sees warranty_collector's conversation
- ❌ Agents can get confused by other agents' messages
- ❌ No way to give an agent just its own conversation context

### The Solution: Message Scoping + State Summaries

The middleware's `_get_recent_messages()` method keeps only the last 2 conversation turns:

```python
# What resolution_specialist actually sees:
[
    SystemMessage("""You are a technical resolution specialist.
    
    Context:
    - Warranty status: in_warranty
    - Issue type: hardware
    
    The customer's warranty status has been confirmed as: in_warranty.
    The issue has been classified as: hardware.
    """),
    HumanMessage("The screen is physically cracked"),
    AIMessage("Tool: record_issue_type"),
    ToolMessage("Recorded issue type"),
    HumanMessage("What should I do?"),  # Current user message
]
```

**Benefits**:
- ✅ Bounded message history (only 2-3 recent turns)
- ✅ Agent sees only its own conversation
- ✅ Context from previous agents available via system message
- ✅ State dict provides structured cross-agent memory
- ✅ Valid message sequences maintained (user/AI/tool alternation)

### Valid Message Sequences

The middleware ensures message histories always follow LLM provider requirements:

**Valid patterns**:
```
System → User → AI
System → User → AI (with tool calls) → Tool → AI → User
```

**Invalid patterns** (prevented by middleware):
```
AI → AI  # Two consecutive AI messages
Tool → User  # Tool message without preceding AI tool call
AI (with tool calls) → User  # Missing tool response
```

The `_get_recent_messages()` method specifically:
1. Collects complete "turns" (user message + AI response + any tool messages)
2. Removes orphaned tool messages without their AI message
3. Ensures the sequence always starts with a user message

### Debugging: Inspecting Agent Views

You can add logging to see what each agent actually sees:

The logging shows:
- Which agent is active
- Exactly what messages that agent sees (after filtering)
- The current state values
- What tools are available to that agent

This makes it easy to verify that:
1. Each agent only sees its recent conversation turns
2. Context from previous agents appears in the system message
3. Message sequences are valid
4. State is correctly passed between agents

In [ ]:
# Enhanced middleware with logging for debugging
class HandoffMiddlewareWithLogging(HandoffMiddleware):
    """Extended middleware that logs what each agent sees."""
    
    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        active_agent = request.state.get("active_agent", "warranty_collector")
        
        print(f"\n{'='*60}")
        print(f"AGENT: {active_agent}")
        print(f"{'='*60}")
        
        # Call parent implementation
        response = super().wrap_model_call(request, handler)
        
        # Log what the agent saw
        print(f"\nMessages sent to {active_agent}:")
        for i, msg in enumerate(request.messages):
            msg_type = type(msg).__name__
            content_preview = str(msg.content)[:100]
            print(f"  {i}. {msg_type}: {content_preview}...")
        
        print(f"\nState: {dict(request.state)}")
        print(f"Tools available: {[t.name for t in request.tools]}")
        
        return response

# Recreate agent with logging middleware
agent_debug = create_agent(
    model,
    tools=all_tools,
    state_schema=SupportState,
    middleware=[HandoffMiddlewareWithLogging(AGENT_CONFIGS)],
    checkpointer=InMemorySaver()
)

# Test with logging
config_debug = {"configurable": {"thread_id": "debug-001"}}

print("Turn 1: Initial message")
result = agent_debug.invoke(
    {"messages": [HumanMessage("My device has an issue")]},
    config_debug
)

## Step 9: Adding Flexibility - Going Back

A key UX requirement is allowing users to correct mistakes. Add "go back" tools:

In [ ]:
@tool
def go_back_to_warranty(
    runtime: ToolRuntime[None, SupportState]
) -> Command:
    """Go back to warranty verification step."""
    return Command(update={
        "active_agent": "warranty_collector",
        "warranty_status": "unknown"  # Reset warranty status
    })

@tool
def go_back_to_classification(
    runtime: ToolRuntime[None, SupportState]
) -> Command:
    """Go back to issue classification step."""
    return Command(update={
        "active_agent": "issue_classifier",
        "issue_type": "unknown"  # Reset issue type
    })

# Add these tools to resolution_specialist configuration
AGENT_CONFIGS["resolution_specialist"].tools.extend([
    go_back_to_warranty,
    go_back_to_classification
])

Update the resolution specialist's prompt to include navigation instructions:

In [ ]:
AGENT_CONFIGS["resolution_specialist"].system_prompt = """You are a technical resolution specialist.

Context:
- Warranty status: {warranty_status}
- Issue type: {issue_type}

Your task:
1. For SOFTWARE issues: provide troubleshooting steps using provide_solution
2. For HARDWARE issues:
   - If IN WARRANTY: explain warranty repair process
   - If OUT OF WARRANTY: escalate_to_human for paid repair options

If the customer indicates any information was wrong, use:
- go_back_to_warranty to correct warranty status
- go_back_to_classification to correct issue type

Be specific and helpful in your solutions."""

Now recreate the agent with the updated configuration:

In [ ]:
# Collect all tools from all configurations (including new go_back tools)
all_tools = []
for config in AGENT_CONFIGS.values():
    all_tools.extend(config.tools)

# Recreate agent with updated tools
agent = create_agent(
    model,
    tools=all_tools,
    state_schema=SupportState,
    middleware=[HandoffMiddleware(AGENT_CONFIGS)],
    checkpointer=InMemorySaver()
)

Test the correction flow:

In [ ]:
# Continue the conversation and test going back
result = agent.invoke(
    {
        "messages": [HumanMessage("Actually, I made a mistake - my device is out of warranty")]
    },
    config
)
print(result["messages"][-1].content)
print(f"Active agent: {result.get('active_agent')}")

## Key Takeaways

The handoffs pattern demonstrates several important concepts:

1. **State machines**: The `active_agent` field creates a state machine where each state has different behavior (prompts + tools).

2. **Dynamic configuration**: Middleware reads state and configures the agent on-the-fly, eliminating the need for separate agent instances.

3. **Message history scoping**: Each agent sees only its recent conversation, preventing unbounded growth and confusion. The middleware:
   - Keeps only the last 2-3 conversation turns for each agent
   - Injects context summaries from state into system messages
   - Ensures valid message sequences (user/AI/tool alternation)

4. **State as cross-agent memory**: The state dict (not message history) is the primary mechanism for sharing information between agents:
   - Structured data (warranty_status, issue_type) persists across handoffs
   - System messages inject state summaries for context
   - Messages remain agent-scoped and ephemeral

5. **State persistence**: A checkpointer maintains state across conversation turns, enabling the handoff mechanism.

6. **Sequential workflows**: Perfect for collecting information in a specific order (warranty → classification → resolution).

7. **Flexibility**: Tools can transition forward OR backward, allowing users to correct mistakes.

### When to use handoffs vs supervisor pattern

Use **handoffs** when:
- Agents need direct conversation with users
- Workflow requires sequential information collection
- State transitions depend on user responses
- Each agent needs its own conversation context

Use **supervisor** when:
- Sub-agents don't need to converse with users
- You want centralized orchestration
- Sub-agents are truly independent specialists

You can also mix both patterns!

### Message History Management Strategy

The key insight: **Separate message history from cross-agent memory**

- **Message history** = Agent-scoped, bounded, ephemeral conversation
- **State dict** = Cross-agent, persistent, structured data
- **System messages** = Bridge between state and agent context

This separation ensures:
- Agents aren't confused by other agents' conversations
- Message histories don't grow unbounded
- Valid message sequences are maintained
- Agents still have full context via state summaries